# decorator walkthrough（细讲版）

## 目标与先修
- 目标：理解 `timed`、`with_tag`、`CountCalls` 的叠加顺序及副作用。
- 先修：理解 Python 函数、闭包、类可调用对象（`__call__`）。


## 流程总览
1. 定义统一 trace 容器。
2. 逐层叠加装饰器并调用函数。
3. 观察调用计数与 trace 事件。
4. 用 `run_demo` 对照模块默认演示。


### 环境检查（离线模块）
- 本步做什么：确认本模块不依赖外部 API key，直接验证本地依赖可导入。
- 为什么这样做：保证 notebook 能在离线环境稳定 Run All。
- 输入：本地 Python 运行环境与项目依赖。
- 输出：导入成功提示。
- 观察点：如果失败，优先检查 `requirements.txt` 和当前虚拟环境。


In [ ]:
from component import CountCalls, run_demo, timed, with_tag

print("环境检查通过：离线依赖已成功导入")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 构建 trace 容器
- 本步做什么：初始化 `trace` 列表，用于记录每层装饰器事件。
- 为什么这样做：装饰器副作用通常隐藏在调用链里，trace 可以显式化。
- 输入：无。
- 输出：空列表 `trace`。
- 观察点：后续每次调用函数都应向该列表追加事件。


In [ ]:
trace = []
print(trace)


### Step 2: 定义并叠加装饰器
- 本步做什么：定义 `sample_total`，叠加 `CountCalls -> with_tag -> timed`。
- 为什么这样做：通过一个具体函数观察“装饰器栈顺序”。
- 输入：`subtotal`、`tax_rate`。
- 输出：带装饰器能力的函数对象。
- 观察点：调用后同时观察返回值、调用次数、trace 事件。


In [ ]:
@CountCalls
@with_tag("billing")
@timed
def sample_total(subtotal: float, tax_rate: float, *, _trace=None) -> float:
    total = round(subtotal * (1.0 + tax_rate), 2)
    if _trace is not None:
        _trace.append(
            {
                "event": "decorated_function_body",
                "subtotal": subtotal,
                "tax_rate": tax_rate,
                "total": total,
            }
        )
    return total

print(sample_total)


### Step 3: 第一次调用并观察副作用
- 本步做什么：执行第一次函数调用。
- 为什么这样做：建立基线，验证返回值与事件记录。
- 输入：`subtotal=100.0, tax_rate=0.13`。
- 输出：总价结果。
- 观察点：检查 `call_count` 是否变为 1，trace 是否记录多个层级事件。


In [ ]:
first_result = sample_total(100.0, 0.13, _trace=trace)
show_kv("第一次调用", {"result": first_result, "call_count": getattr(sample_total, 'call_count', None)})
show_json(trace)


### Step 4: 第二次调用并验证计数增长
- 本步做什么：执行第二次调用。
- 为什么这样做：验证 `CountCalls` 的累计行为，而非单次行为。
- 输入：`subtotal=80.0, tax_rate=0.13`。
- 输出：第二次调用结果。
- 观察点：`call_count` 应递增，trace 事件数量应增加。


In [ ]:
second_result = sample_total(80.0, 0.13, _trace=trace)
show_kv(
    "第二次调用",
    {
        "result": second_result,
        "call_count": getattr(sample_total, "call_count", None),
        "trace_len": len(trace),
    },
)
show_json(trace)


## 端到端结果

### Step 5: 对照模块默认 run_demo 输出
- 本步做什么：调用 `run_demo()`。
- 为什么这样做：确认手动构造示例与模块内置演示输出一致。
- 输入：无。
- 输出：`demo_result`（含 final_answer 与 trace）。
- 观察点：比较你手动 trace 与模块 trace 的事件语义。


In [ ]:
demo_result = run_demo()
show_json(demo_result)


## 常见错误
- 装饰器顺序写反：会导致 trace 顺序与计数行为变化。
- 忘记传 `_trace`：教学时就看不到中间过程。
- 误以为 `CountCalls` 只统计函数体调用：实际上统计的是装饰后函数调用次数。

## 总结
- 装饰器本质是“可组合的行为包装层”。
- 教学调试要抓住三件事：返回值、调用计数、trace 序列。
